In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_community = 3 # Set the index j for number of community(lvd 1900, Vertex, Pala Mesa)
number_of_labor = 4 # Set the index i for number of hire(A, B, C, D)
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
D = [26, 30, 28] # Demand of Order from each community 
labor_cost = [90, 80, 82, 88]  #  Fixed cosy for hiring each labor 
total_labor = 2 # At most 2 people can be hired
unit_cost = [[2, 1.6, 2.4], [1.4, 2.3, 2], [2.1, 1.7, 1.9], [1.8, 2.5, 1.8]]   # Unit Cost to deliver one order by person i to Community j
demand = [250, 170]  # Demand to produce each type of juice
weekly_capacity = 45   # Each employee could delivery maximum 45 orders per week


In [4]:
# Set the Variable list: Set whether person i should be hired as x; Set number of order to deliver by person  i to community j as nx
# Set the variable x to binary number, nx to integer number

x = [] 
for i in range(number_of_labor):
    x.append(m.addVar(vtype=GRB.BINARY, name='x{}'.format(i + 1)))

nx = [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
for i in range(number_of_labor):
    for j in range(number_of_community):
        nx[i][j] = m.addVar(vtype=GRB.INTEGER, name='nx{}'.format(str(i + 1) + str(j + 1)))

In [5]:
# Set the Minimize Obijective: Total Cost
m.setObjective(quicksum([sum([unit_cost[i][j] * nx[i][j] for j in range(number_of_community)]) for i in range(number_of_labor)]) + quicksum([(x[i] * labor_cost[i]) for i in range(number_of_labor)]) ,  GRB.MINIMIZE)

In [6]:
# Set Non Negative number of order
for i in range(number_of_labor):
     for j in range(number_of_community):
            c1 = m.addConstr(nx[i][j]  >= 0)

In [7]:
print(D)

[26, 30, 28]


In [8]:
# Satisfy demand of a community
c2 = []
for j in range(number_of_community):
    c2.append(m.addConstr(sum([nx[i][j] for i in range(number_of_labor)]) >= D[j]))

In [9]:
# No delivery allowed from no-hired people
c3 = []
for i in range(number_of_labor):
    c3.append(m.addConstr(sum([nx[i][j] for j in range(number_of_community)]) <= (x[i] * weekly_capacity )))

In [10]:
# If A is hired, B must be hired
c4 = m.addConstr(x[0] <= x[1])


In [11]:
# At most 2 people can be hired
c5 = m.addConstr(sum([x[i] for i in range(number_of_labor)]) <= total_labor)

In [12]:
# Either B or D must always be hired
c6 = m.addConstr((x[1] + x[3]) >= 1)

In [13]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 22 rows, 16 columns and 48 nonzeros
Model fingerprint: 0x61c572dd
Variable types: 0 continuous, 16 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [1e+00, 9e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 315.9000000
Presolve removed 12 rows and 0 columns
Presolve time: 0.00s
Presolved: 10 rows, 16 columns, 36 nonzeros
Variable types: 0 continuous, 16 integer (4 binary)

Root relaxation: objective 2.931444e+02, 12 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  293.14444    0    3  315.90000  293.14444  7.20%     -    0s
H    0     0                     304.4000000  293.14444  3.70%     -    0s
H    0     0                     303.9000000  293.14444  3.54%     - 

In [16]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
          x2            1 
          x3            1 
        nx21           26 
        nx23           13 
        nx32           30 
        nx33           15 


In [17]:
# Get the Optimal Objective Value
m.ObjVal

303.9